# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [13]:
# Write your code below.
from dotenv import load_dotenv
import os
load_dotenv()

True

In [14]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [15]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [16]:
# Write your code below.

dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

# adding lags for variable Close
dd_lags_close = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))  
)
# adding lags for variacle Adj Close
dd_lags_AdjClose = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(AdjClose_lag_1 = x['Adj Close'].shift(1))  
)
# adding returns for variable Close
dd_rets1 = dd_lags_close.assign(
    Close_returns = lambda x: x['Close'] / x['Close_lag_1'] - 1  
)
# adding returns for variable Adj Close
dd_rets2 = dd_lags_AdjClose.assign(
    Adj_Close_returns = lambda x: x['Adj Close'] / x['AdjClose_lag_1'] - 1  
)
# adding hi_lo_range
dd_lags_hi_lo = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(hi_lo_range = x['High'] - x['Low'])  
)

dd_feat = dd.concat([dd_rets1, dd_rets2[['Adj_Close_returns']], dd_lags_hi_lo[['hi_lo_range']]], axis=1)




/var/folders/rp/4_wz6kc51h95rr7rqf2zlns80000gn/T/ipykernel_3914/97323550.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lags_close = dd_px.groupby('Ticker', group_keys=False).apply(
/var/folders/rp/4_wz6kc51h95rr7rqf2zlns80000gn/T/ipykernel_3914/97323550.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lags_AdjClose = dd_px.groupby('Ticker', group_keys=False).apply(
/var/folders/rp/4_wz6kc51h95rr7rqf2zlns80000gn/T/ipykernel_3914/97323550.py:22: UserWarning: `meta` is not specified, inferred from partia

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [23]:
import dask.dataframe as dd

# Function to apply rolling mean on each partition
def rolling_mean_partition(df):
    # Ensure that 'returns' column exists before applying rolling
    df['returns'] = df['Close_returns'].combine_first(df['Adj_Close_returns'])
    
    # Apply rolling mean on the 'returns' column
    df['returns_ma'] = df['returns'].rolling(10, min_periods=1).mean()
    return df

# Apply the function to each partition using map_partitions
dd_feat = dd_feat.map_partitions(rolling_mean_partition)

# Compute the final DataFrame
dd_feat_pd = dd_feat.compute()

# Print the result
print(dd_feat_pd.head())


Price        Date   Adj Close       Close        High         Low        Open  \
Ticker                                                                          
DOV    2024-01-02  149.826508  151.520004  154.050003  150.979996  153.169998   
DOV    2024-01-03  146.672165  148.330002  150.399994  147.690002  150.330002   
DOV    2024-01-04  146.444733  148.100006  149.380005  147.940002  148.699997   
DOV    2024-01-05  145.841553  147.490005  148.369995  146.850006  147.500000   
DOV    2024-01-08  146.573273  148.229996  148.339996  145.839996  146.559998   

Price      Volume  Year  Close_lag_1  Close_returns  Adj_Close_returns  \
Ticker                                                                   
DOV      695400.0  2024          NaN            NaN                NaN   
DOV      844800.0  2024   151.520004      -0.021053          -0.021053   
DOV      873400.0  2024   148.330002      -0.001551          -0.001551   
DOV      878500.0  2024   148.100006      -0.004119          -

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
 No, Dask can handle rolling windows and moving averages, just like Pandas. We only need to switch to Pandas if we're working with a small dataset that fits in memory.

+ Would it have been better to do it in Dask? Why?
Since the dataset is large, using Dask might be better because it manages memory more efficiently and processes data only when needed.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.